# Train Second Model: Linear SVM

In [1]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?code_challenge=v2pAz1K0SpBbeMVoilQ-BTvGco_67TpSJ8ib0tnLEAA&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [galli.giuly@gmail.com].
Your current project is [reddit-master].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [4]:
!gcloud config set project reddit-master

Updated property [core/project].


In [6]:
import pandas as pd
import numpy as np
import pickle as pkl
import nltk.data
import joblib
import nltk
import ast
import logging
nltk.download('punkt')

import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from gensim.models import FastText
from nltk.tokenize import TweetTokenizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import LinearSVC, SVC

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/giuliagalli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
subreddits = ['aww', 'nba', 'movies', 'todayilearned', 'IAmA', 'Fitness', 'worldnews', 'technology', 'europe', 'politics', 'atheism','science', 'funny', 'gaming']

In [14]:
%%time

model_SVM_df = pd.DataFrame(columns = ["subreddit", "body", "subreddit_id"])

for subreddit in subreddits:
    !/Users/giuliagalli/google-cloud-sdk/bin/gsutil cp gs://reddit_final_results/comments_posts_{subreddit}.pkl .
    print("comments_posts_" + subreddit + ".pkl downloaded")
    
    df = pd.read_pickle("comments_posts_" + subreddit + ".pkl")

    model_SVM_df = pd.concat([model_SVM_df, df], ignore_index=True)

Copying gs://reddit_final_results/comments_posts_aww.pkl...
\ [1 files][ 63.8 MiB/ 63.8 MiB]    5.9 MiB/s                                   
Operation completed over 1 objects/63.8 MiB.                                     
comments_posts_aww.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


Copying gs://reddit_final_results/comments_posts_nba.pkl...
/ [1 files][ 88.3 MiB/ 88.3 MiB]    5.9 MiB/s                                   
Operation completed over 1 objects/88.3 MiB.                                     
comments_posts_nba.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


Copying gs://reddit_final_results/comments_posts_movies.pkl...
- [1 files][103.2 MiB/103.2 MiB]    6.2 MiB/s                                   
Operation completed over 1 objects/103.2 MiB.                                    
comments_posts_movies.pkl downloaded


...

Copying gs://reddit_final_results/comments_posts_funny.pkl...
| [1 files][ 69.3 MiB/ 69.3 MiB]    5.8 MiB/s                                   
Operation completed over 1 objects/69.3 MiB.                                     
comments_posts_funny.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


Copying gs://reddit_final_results/comments_posts_gaming.pkl...
- [1 files][ 79.4 MiB/ 79.4 MiB]    6.5 MiB/s                                   
Operation completed over 1 objects/79.4 MiB.                                     
comments_posts_gaming.pkl downloaded


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


CPU times: user 2min 40s, sys: 4min 9s, total: 6min 49s
Wall time: 14min 16s


In [10]:
model_SVM_df.shape

(10833350, 4)

In [18]:
X = model_SVM_df['body']
y = model_SVM_df['subreddit_id']

# Definint a fucntion that slit the dataset into three subsets: train, val and test
def train_dev_test_split(X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42, stratify=y_val)
    return dict(X_train=X_train, 
                X_val=X_val, 
                X_test=X_test, 
                y_train=y_train.astype('int'), 
                y_val=y_val.astype('int'), 
                y_test=y_test.astype('int'))

In [19]:
%%time

model_SVM_df_split = train_dev_test_split(X,y)

CPU times: user 32.9 s, sys: 9.53 s, total: 42.4 s
Wall time: 44.5 s


In [20]:
model_SVM_df_split.keys()

dict_keys(['X_train', 'X_val', 'X_test', 'y_train', 'y_val', 'y_test'])

In [21]:
model_SVM_df_split['y_train'].unique()

array([ 0,  1, 13, 10,  3, 12,  5,  7,  8,  4,  2,  9,  6, 11])

In [25]:
%%time

model_svm = Pipeline([
            ('tfidf', TfidfVectorizer(max_df=0.5, ngram_range=(1,2), preprocessor=' '.join)),
            ('clf', LinearSVC(), n_jobs=1),
            ])

model_svm.fit(model_SVM_df_split['X_train'], model_SVM_df_split['y_train'])

CPU times: user 1h 48min 35s, sys: 49min 45s, total: 2h 38min 21s
Wall time: 4h 17min 22s


In [32]:
import joblib

joblib.dump(model_svm, "model_svm.joblib")


['model_svm.joblib']

In [52]:
!/Users/giuliagalli/google-cloud-sdk/bin/gsutil cp model_svm.joblib gs://reddit_models/

Copying file://model_svm.joblib [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [1 files][  4.6 GiB/  4.6 GiB]    6.5 MiB/s                                   
Operation completed over 1 objects/4.6 GiB.                                      


In [48]:
%%time

prediction = model_svm.predict(model_SVM_df_split['X_val'])

CPU times: user 4min 41s, sys: 7min 1s, total: 11min 43s
Wall time: 18min 41s


In [49]:
%%time

print(classification_report(model_SVM_df_split['y_val'], prediction))


              precision    recall  f1-score   support

           0       0.76      0.81      0.78    116144
           1       0.40      0.34      0.37    116121
           2       0.53      0.55      0.54    116040
           3       0.49      0.61      0.54    116073
           4       0.52      0.53      0.53    116004
           5       0.28      0.27      0.27    115905
           6       0.55      0.54      0.55    116061
           7       0.58      0.61      0.59    116086
           8       0.71      0.72      0.72    116013
           9       0.48      0.50      0.49    116052
          10       0.46      0.50      0.48    116360
          11       0.47      0.48      0.47    116066
          12       0.30      0.23      0.26    115999
          13       0.35      0.30      0.33    116078

   micro avg       0.50      0.50      0.50   1625002
   macro avg       0.49      0.50      0.49   1625002
weighted avg       0.49      0.50      0.49   1625002

CPU times: user 3.43 s, s

### Testing the model

In [38]:
# funny => wrong

sentence1 = "I designed the Family Feud Fender to quickly remove yourself from family holiday drama."

In [39]:
tokenizer = TweetTokenizer()

text1 = tokenizer.tokenize(sentence1)

result1 = model_svm.predict([text1])

print(result1)

[4]


In [40]:
model_SVM_df[model_SVM_df['subreddit_id'] == int(result1)].head()

,Unnamed: 0.1,body,subreddit,subreddit_id
6190536,49079.0,"[kind, brazilian, move]",europe,4
6190537,49080.0,"[trulli, consensu, facto, legal, europa, itali...",europe,4
6190538,49081.0,"[dutch, articl, day, ago, link, quot, raad, va...",europe,4
6190539,49082.0,"[peru, recent, trust, currenc, , major, trans...",europe,4
6190540,49083.0,"[argument, actual, talk, 1930, voter, afraid, ...",europe,4


In [24]:
# nba => wrong

sentence2 = "It's like feast or famine. It seemed like there were 6 games yesterday and only 2 today and I think tomorrow there is around 6"

In [25]:
tokenizer = TweetTokenizer()

text2 = tokenizer.tokenize(sentence2)

result2 = model_svm.predict([text2])

print(result2)

[2]


In [26]:
model_SVM_df[model_SVM_df['subreddit_id'] == int(result2)].head()

,Unnamed: 0.1,body,subreddit,subreddit_id
7737574,88650.0,"[argu, deep, human, except, thing, defianc, idea]",atheism,2
7737575,88651.0,"[differ, harri, potter, bibl, coupl, billion, ...",atheism,2
7737576,88652.0,"[explain, argument]",atheism,2
7737577,88653.0,"[driven, selfish]",atheism,2
7737578,88654.0,"[hear, god, heal, , sarcasm]",atheism,2


In [33]:
# movies => correct

sentence3 = "Netflix takes over lease to iconic but failed 71-year-old movie theater in New York City to show its own original films."

In [34]:
tokenizer = TweetTokenizer()

text3 = tokenizer.tokenize(sentence3)

result3 = model_svm.predict([text3])

print(result3)

[7]


In [35]:
model_SVM_df[model_SVM_df['subreddit_id'] == int(result3)].head()

,Unnamed: 0.1,body,subreddit,subreddit_id
1547237,13233.0,"[13, minut, snake, eye]",movies,7
1547238,13234.0,"[14, , grand, budapest, hotel15, , mad, max,...",movies,7
1547239,13235.0,"[took, 8, peopl, help, ocean, heist]",movies,7
1547240,13236.0,"[remindm, 6, month]",movies,7
1547241,13237.0,"[mean, lot, defend, way, defend, attack, negat...",movies,7


In [124]:
# movies => correct

sentence3 = "Netflix takes over lease to iconic but failed 71-year-old movie theater in New York City to show its own original films."

,Unnamed: 0.1,body,subreddit,subreddit_id
1547237,13233.0,"[13, minut, snake, eye]",movies,7
1547238,13234.0,"[14, , grand, budapest, hotel15, , mad, max,...",movies,7
1547239,13235.0,"[took, 8, peopl, help, ocean, heist]",movies,7
1547240,13236.0,"[remindm, 6, month]",movies,7
1547241,13237.0,"[mean, lot, defend, way, defend, attack, negat...",movies,7


In [41]:
# funny => correct

sentence4 = "Making Friends Monday! Share your game tags here!"

In [42]:
tokenizer = TweetTokenizer()

text4 = tokenizer.tokenize(sentence4)

result4 = model_svm.predict([text4])

print(result4)

[6]


In [43]:
model_SVM_df[model_SVM_df['subreddit_id'] == int(result4)].head()

,Unnamed: 0.1,body,subreddit,subreddit_id
10059610,477973.0,"[sad, gotten, point, fan, better, game, aaa, s...",gaming,6
10059611,477974.0,"[pencil, refer]",gaming,6
10059612,477975.0,"[nake, gun, 33, 13, mayb]",gaming,6
10059613,477976.0,"[knight, old, republ, game]",gaming,6
10059614,477977.0,"[seen, redead]",gaming,6
